In [33]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This file creates figures displaying NOT narrowband
# filters covering Lya for lensed LAE candidates in eBOSS
# identified by Cao et al. 2020 that the Nordic Optical
# Telescope can observe in February 2023.

In [34]:
import os
import glob

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

In [2]:
def plot():

    '''
    Plots NOT narrowband filters covering Lya at the redshift of the 
    LAEs
    '''

    # Set home directory
    home = os.getcwd()

    # SDSS IDs of the targets
    targets = np.array(['6459-56273-547','6020-56087-977','3973-55323-667',
        '4723-56033-445','3676-55186-623','5004-55711-151',
        '6967-56447-801','3865-55272-67','5782-56272-549',
        '6732-56370-859','3945-55648-275','5403-55979-51',
        '3691-55274-138','4725-55711-295','7120-56720-417',
        '5369-56272-541','3876-55245-223','5980-56338-72',
        '7096-56683-470','5172-56071-193','5168-56035-693',
        '4644-55922-885','6315-56181-654','5164-56067-735',
        '6408-56329-327','6785-56487-647','6833-56413-181',
        '4473-55589-231','6316-56483-84','6422-56328-303',
        '6014-56072-764','4446-55589-635','5359-55953-51',
        '6629-56365-311','4756-55631-615','10751-58220-189',
        '4707-55653-211','6805-56442-57','5184-56352-669',
        '4651-56008-17','4614-55604-647','6018-56067-570'], dtype=str)
    
    # Redshifts of the targets, as determined by Gaussian fits to 
    # nebular emission lines
    z = np.array([2.1009,2.1124,2.1436,2.1451,2.1817,
        2.193,2.1933,2.1995,2.205,2.2302,
        2.2526,2.2765,2.3758,2.377,2.4079,
        2.4495,2.4956,2.53,2.5997,2.6524,
        2.6672,2.679,2.6833,2.6892,2.7272,
        2.7284,2.7301,2.7536,2.7539,2.7598,
        2.7907,2.7925,2.8023,2.9063,2.9296,
        2.9349,2.8231,2.7901,2.6479,2.6472,
        2.2212,2.5286], dtype=np.float64)

    # Files containing the narrowband filter throughputs
    filter_files = glob.glob(home + '/data/filters/*.txt')

    # For each target
    for i, target in enumerate(targets):

        plt.close('all')

        # This counter counts how many filters target Lya
        k = 0

        # For each filter
        for j, file in enumerate(filter_files):

            # If the filter is one of #129, #130, or #131
            # (necessary since the files use different delimiters)
            if any(e in file for e in ['129','130','131']):

                # Extract the data from the file
                w, t = np.loadtxt(file, delimiter='  ', comments='#', unpack=True)

            else:

                # Extract the data from the file
                w, t = np.loadtxt(file, delimiter=' ', comments='#', unpack=True)

            # If the file uses nm for wavelength
            if any(e in file for e in ['19','23','24','30','31','32','33','34','35','37','39','79','88','99']):
                
                # Convert wavelength to Angstroms
                w = w * 10

            # If the file measures transmission as a ratio
            # and not a percentage
            if all(t <= 1):

                # Convert transmission to a percentage
                t = t * 100

            # If any part of the filter has a transmission more than 5% between
            # 1210-1221 A at this redshift
            if any(t[(w >= 1210 * (1 + z[i])) & (w <= 1221 * (1 + z[i]))] >= 5):

                # If this is the first filter targeting Lya at the target's redshift
                if k == 0:

                    fig, ax = plt.subplots(constrained_layout=True)
                    
                    # Set x-axis limits
                    x_lim = (w[0] / (1 + z[i]), w[-1] / (1 + z[i]))

                # If the minimum wavelength of the filter is less than the current
                # lower bound for the x-axis
                if w[0] / (1 + z[i]) < x_lim[0]:

                    # Set the lower bound for the x-axis to the filter's minimum wavelength
                    x_lim[0] == w[0]

                # If the maximum wavelength of the filter is less than the current
                # lower bound for the x-axis
                if w[-1] / (1 + z[i]) > x_lim[1]:

                    # Set the lower bound for the x-axis to the filter's maximum wavelength
                    x_lim[1] == w[-1]

                # Plot a shaded transmission curve of the filter in the rest wavelength at the
                # target's redshift
                ax.fill_between(w / (1 + z[i]), t, y2=0, alpha=0.5)
                ax.plot(w / (1 + z[i]), t, label=os.path.basename(file)[:-4])

                k = k + 1

        # If at least 1 filter covers Lya at the target's redshift
        if k != 0:

            # Plot a vertical line corresponding to the rest Lya wavelength
            ax.axvline(1215.67, ls='dashed', c='black')

            # Set the data limits of the axes
            ax.set_xlim(*x_lim)
            ax.set_ylim(bottom=0)

            # Add a legend for the filter IDs
            ax.legend(loc='upper right')

            # Label the axes
            fig.supxlabel(r'Rest wavelength ($\rm{\AA}$)')
            fig.supylabel('Transmission (%)')

            # Annotate the SDSS ID in the top left corner
            at = AnchoredText(target, loc='upper left', frameon=False)
            ax.add_artist(at)

            # Save the figure
            fig.savefig(home + '/figs/' + target + '.pdf', bbox_inches='tight')


In [36]:
plot()